> ### Pipeling to Merge Sequencing Libraries from Different Sequencing Runs that have been denoised by DADA2  and Finish with the Standard Processing Pipeline###
* Merge multiple libraries for the *SAME* sequencing region (i.e. ITS+ITS... or SSU+SSU...)
* Complete remainder of processing pipieline: prepare Rep Seqs, Classify Seqs, Make Tree, Make Phyloseq Object

*Informed by the "Fecal Microbiota Transplant Tutorial" for QIIME2*

### Commands to Install Dependencies ####
##### || QIIME2 ||
* conda create -n qiime2-2018.2 --file https://data.qiime2.org/distro/core/qiime2-2018.2-conda-linux-64.txt  

* source activate qiime2-2018.2

##### || Copyrighter rrn Database ||
* The script will automatically install the curated GreenGenes rrn attribute database
* https://github.com/fangly/AmpliCopyrighter

##### || rpy2 (don't use conda version) ||
* pip install rpy2  

##### || phyloseq ||
* conda install -c r-igraph 
* Rscript -e "source('http://bioconductor.org/biocLite.R');biocLite('phyloseq')" 

##### || R packages ||
* ape   (natively installed with in conda environment)


### Citations ###
* Caporaso, J. G., Kuczynski, J., Stombaugh, J., Bittinger, K., Bushman, F. D., Costello, E. K., *et al.* (2010). QIIME allows analysis of high-throughput community sequencing data. Nature methods, 7(5), 335-336.


* McMurdie and Holmes (2013) phyloseq: An R Package for Reproducible Interactive Analysis and Graphics of Microbiome Census Data. PLoS ONE. 8(4):e61217


* Paradis E., Claude J. & Strimmer K. 2004. APE: analyses of phylogenetics and evolution in R language. Bioinformatics 20: 289-290.


* Angly, F. E., Dennis, P. G., Skarshewski, A., Vanwonterghem, I., Hugenholtz, P., & Tyson, G. W. (2014). CopyRighter: a rapid tool for improving the accuracy of microbial community profiles through lineage-specific gene copy number correction. Microbiome, 2(1), 11.


###### Last Modified by R. Wilhelm on October 12th, 2017 ######


# Step 1: User Input

In [ ]:
import os, re

# Provide the directory where files are located 
#directory = '/home/user/PROJECT/16S/'
directory = ''

# Provide a list of all the FeatureTables you will merge
# Produced by QIIME2 in STEP 7 (i.e. DADA2 Denoising/Merging/FeatureTable)
#qva_files = ['SSU.table.Library.1.qza','SSU.table.Library.2.qza', ..., 'SSU.table.Library.n.qza']
qva_files = ['','','']

# Provide a list of all the Representative Sequences you will merge
# Also produced from STEP 7 (i.e. DADA2 Denoising/Merging/FeatureTable)
#seq_files = ['SSU.rep.seqs.Library.1.qza','SSU.rep.seqs.Library.2.qza', ..., 'SSU.rep.seqs.Library.n.qza']
seq_files = ['','','']

# Provide a concatenated metadatafile with matching column order
#metadata = 'SSU.metadata.combined.tsv'
metadata = ''

## Enter Minimum Support for Keeping QIIME Classification
# Note: Classifications that do not meet this criteria will simply be retained, but labeled 'putative'
min_support = 0.8

domain = 'bacteria'   # 'bacteria' | 'fungi'


##################################    IMPORTANT    #####################################
##  DO NOT give your samples names that begin with a digit. It will break this script ##
##  at the import to phyloseq stage. It doesn't ruin anything, but makes it a pain.   ##
########################################################################################

# Step 2: Merge Feature Tables using QIIME

In [ ]:
%mkdir $directory/output

combined_qva = [m+" "+n for m,n in zip(["--i-tables"]*len(qva_files),qva_files)]

os.system(' '.join([
    "qiime feature-table merge",
    ' '.join(combined_qva),
    "--o-merged-table "+directory+"/output/merged.table.final.qza"
]))


combined_seq = [m+" "+n for m,n in zip(["--i-data"]*len(seq_files),seq_files)]

os.system(' '.join([
    "qiime feature-table merge",
    ' '.join(combined_seq),
    "--o-merged-table "+directory+"/output/merged.table.final.qza"
]))

# Step 3: Create Summary of OTUs

In [ ]:
os.system(' '.join([
    "qiime feature-table summarize",
    "--i-table "+directory+"/output/merged.table.final.qza",
    "--o-visualization "+directory+"/output/merged.table.final.qzv",
    "--m-sample-metadata-file "+directory+metadata
]))

os.system(' '.join([
    "qiime feature-table tabulate-seqs",
    "--i-data "+directory+"/output/merged.rep.seqs.final.qza",
    "--o-visualization "+directory+"/output/merged.rep.seqs.final.qzv"
])) 

# Step 4: Make Phylogenetic Tree

In [ ]:
if domain != "fungi":
    # Generate Alignment with MAFFT
    os.system(' '.join([
        "qiime alignment mafft",
        "--i-sequences "+directory+"/output/merged.rep.seqs.final.qza",
        "--o-alignment "+directory+"/output/merged.rep.seqs.aligned.qza"
    ]))

    # Mask Hypervariable parts of Alignment
    os.system(' '.join([
        "qiime alignment mask",
        "--i-alignment "+directory+"/output/merged.rep.seqs.aligned.qza",
        "--o-masked-alignment "+directory+"/output/merged.rep.seqs.aligned.masked.qza"
    ])) 

    # Generate Tree with FastTree
    os.system(' '.join([
        "qiime phylogeny fasttree",
        "--i-alignment "+directory+"/output/merged.rep.seqs.aligned.masked.qza",
        "--o-tree "+directory+"/output/merged.rep.seqs.tree.unrooted.qza"
    ])) 

    # Root Tree
    os.system(' '.join([
        "qiime phylogeny midpoint-root",
        "--i-tree "+directory+"/output/merged.rep.seqs.tree.unrooted.qza",
        "--o-rooted-tree "+directory+"/output/merged.rep.seqs.tree.final.qza"
    ])) 


# Step 5: Classify Seqs

In [ ]:
# Classify
if domain == 'bacteria':
    os.system(' '.join([
        "qiime feature-classifier classify-sklearn",
        "--i-classifier /home/db/GreenGenes/qiime2_13.8.99_515.806_nb.classifier.qza",
        "--i-reads "+directory+"/output/merged.rep.seqs.final.qza",
        "--o-classification "+directory+"/output/merged.taxonomy.final.qza"
    ]))

if domain == 'fungi':
    os.system(' '.join([
        "qiime feature-classifier classify-sklearn",
        "--i-classifier /home/db/UNITE/qiime2_unite_ver7.99_20.11.2016_classifier.qza",
        "--i-reads "+directory+"/output/merged.rep.seqs.final.qza",
        "--o-classification "+directory+"/output/merged.taxonomy.final.qza"
    ]))

# Output Summary
os.system(' '.join([
    "qiime metadata tabulate",
    "--m-input-file "+directory+"/output/merged.taxonomy.final.qza",
    "--o-visualization "+directory+"/output/merged.taxonomy.final.summary.qzv"
])) 

# Step 6: Prepare Data for Import to Phyloseq

In [ ]:
## Make Function to Re-Format Taxonomy File to Contain Full Column Information 
# and factor in the certain of the taxonomic assignment

def format_taxonomy(tax_file, min_support):
    output = open(re.sub(".tsv",".fixed.tsv",tax_file), "w")
    output.write("\t".join(["OTU","Domain","Phylum","Class","Order","Family","Genus","Species"])+"\n")
    
    with open(tax_file, "r") as f:
        next(f) #skip header

        for line in f:
            line = line.strip()
            line = line.split("\t")

            read_id = line[0]
            tax_string = line[1]

            # Annotate those strings which do not meet minimum support
            if float(line[2]) < float(min_support):
                tax_string = re.sub("__","__putative ",tax_string)

            # Remove All Underscore Garbage (gimmie aesthetics)
            tax_string = re.sub("k__|p__|c__|o__|f__|g__|s__","",tax_string) 

            # Add in columns containing unclassified taxonomic information
            # Predicated on maximum 7 ranks (Domain -> Species)
            full_rank = tax_string.split(";")
            last_classified = full_rank[len(full_rank)-1]

            count = 1
            while last_classified == " ":
                last_classified = full_rank[len(full_rank)-count]
                count = count + 1


            for n in range(full_rank.index(last_classified)+1, 7, 1):
                try:
                    full_rank[n] = "unclassified "+last_classified
                except:
                    full_rank.append("unclassified "+last_classified)

            output.write(read_id+"\t"+'\t'.join(full_rank)+"\n")
            
    return()

In [ ]:
#####################
## Export from QIIME2

## Final Output Names
fasta_file = directory+"/output/merged.rep.seqs.final.fasta"
tree_file = directory+"/output/merged.tree.final.nwk"
tax_file = directory+"/output/merged.taxonomy.final.tsv"
count_table = directory+"/output/merged.counts.final.biom"

# Export Classifications
os.system(' '.join([
    "qiime tools export",
    directory+"/output/merged.taxonomy.final.qza",
    "--output-dir "+directory+"/output/"
]))

# Reformat Classifications to meet phyloseq format
format_taxonomy(directory+"/output/taxonomy.tsv", min_support)

# Export SV Table
os.system(' '.join([
    "qiime tools export",
    directory+"/output/merged.table.final.qza",
    "--output-dir "+directory+"/output/"
]))

# Export SV Sequences
os.system(' '.join([
    "qiime tools export",
    directory+"/output/merged.rep.seqs.final.qza",
    "--output-dir "+directory+"/output/"
]))

if domain == "bacteria":
    # Export Tree
    os.system(' '.join([
        "qiime tools export",
        directory+"/output/merged.rep.seqs.tree.final.qza",
        "--output-dir "+directory+"/output/"
    ]))

    %mv $directory/output/tree.nwk $tree_file

# Rename Exported Files
%mv $directory/output/dna-sequences.fasta $fasta_file
%mv $directory/output/feature-table.biom $count_table
%mv $directory/output/taxonomy.fixed.tsv $tax_file

# Step 7: Get 16S rRNA Gene Copy Number (rrn)

In [ ]:
## This step is based on the database contructed for the software 'copyrighter'
## The software itself lacked information about datastructure (and, the import of a biom from QIIME2 failed, likely because there are multiple versions of the biom format)

if domain == 'bacteria':
    ## Download copyrighter database
    !git clone https://github.com/fangly/AmpliCopyrighter $directory/temp/
        
    ## There are multiple GreenGenes ID numbers for a given taxonomic string.
    ## However, the copyrighter database uses the same average rrn copy number.
    ## We will therefore just use the taxonomic strings, since QIIME2 does not output the ID numbers

    !sed -e '1,1075178d; 1078115d' $directory/temp/data/201210/ssu_img40_gg201210.txt > $directory/output/copyrighter.tax.strings.tsv

    ## Create Dictionary of rrnDB
    rrnDB = {}

    with open(directory+"/output/copyrighter.tax.strings.tsv", "r") as f:
        for line in f:
            line = line.strip()
            line = line.split("\t")

            try:
                rrnDB[line[0]] = line[1]
                
            except:
                pass

    ## Attribute rrn to readID from taxonomy.tsv
    output = open(directory+"/output/merged.seqID.to.rrn.final.tsv","w")
    output.write("Feature ID\trrn\n")

    with open(directory+"/output/taxonomy.tsv", "r") as f:
        missing = 0
        total = 0
        next(f)  # Skip Header

        for line in f:
            line = line.strip()
            line = line.split("\t")

            seqID = line[0]

            try:
                rrn = rrnDB[line[1]]
                
            except:
                rrn = "NA"
                missing = missing + 1

            total = total + 1
            output.write(seqID+"\t"+rrn+"\n")

    print("\nPercent of OTUs Missing {:.1%}".format(float(missing)/total))
    print("Don't Panic! The majority of missing OTUs could be low abundance.")

# Step 8: Import into Phyloseq

In [ ]:
## Setup R-Magic for Jupyter Notebooks
import rpy2
%load_ext rpy2.ipython

def fix_biom_conversion(file):
    with open(file, 'r') as fin:
        data = fin.read().splitlines(True)
    with open(file, 'w') as fout:
        fout.writelines(data[1:])

In [ ]:
import pandas as pd
%R library(phyloseq)
%R library(ape)

#### IMPORT DATA to R
## For '.tsv' files, use Pandas to create a dataframe and then pipe that to R
## For '.biom' files, first convert using 'biom convert' on the command-line
## Had problems importing the count table with pandas, opted for using read.table in R

# Import Taxonomy File
tax_file = pd.read_csv(directory+"/output/merged.taxonomy.final.tsv", sep="\t")
%R -i tax_file
%R rownames(tax_file) = tax_file$OTU
%R tax_file$OTU <- NULL
%R tax_file <- tax_file[sort(row.names(tax_file)),] #read names must match the count_table

# Import Sample Data
sample_file = pd.read_csv(directory+metadata, sep="\t", keep_default_na=False)
%R -i sample_file
%R rownames(sample_file) = sample_file$X.SampleID   
%R sample_file$X.SampleID <- NULL
%R sample_file$LinkerPrimerSequence <- NULL  ## Clean-up some other stuff

# Import Count Data
os.system(' '.join([
    "biom convert",
    "-i",
    directory+"/output/merged.counts.final.biom",
    "-o",
    directory+"/output/merged.counts.final.tsv",
    "--to-tsv"
]))

# The biom converter adds a stupid line that messes with the table formatting
fix_biom_conversion(directory+"/output/merged.counts.final.tsv")

# Finally import
count_table = pd.read_csv(directory+"/output/merged.counts.final.tsv", sep="\t")
%R -i count_table
%R rownames(count_table) = count_table$X.OTU.ID   
%R count_table$X.OTU.ID <- NULL    
%R count_table <- count_table[sort(row.names(count_table)),] #read names must match the tax_table

# Convert to Phyloseq Objects
%R p_counts = otu_table(count_table, taxa_are_rows = TRUE)    
%R p_samples = sample_data(sample_file)    
%R p_tax = tax_table(tax_file)
%R taxa_names(p_tax) <- rownames(tax_file) # phyloseq throws out rownames
%R colnames(p_tax) <- colnames(tax_file) # phyloseq throws out colnames

# Merge Phyloseq Objects
%R p = phyloseq(p_counts, p_tax)

if domain == "bacteria":
    # Import Phylogenetic Tree
    tree_file = directory+"/output/merged.tree.final.nwk"
    %R -i tree_file  
    %R p_tree <- read.tree(tree_file)

    # Combine All Objects into One Phyloseq
    %R p_final <- merge_phyloseq(p, p_samples, p_tree)

else:
    # Combine All Objects into One Phyloseq
    %R p_final <- merge_phyloseq(p, p_samples)
    
# Save Phyloseq Object as '.rds'
output = directory+"/output/p_merged.final.rds"
%R -i output
%R saveRDS(p_final, file = output)

# Confirm Output
%R print(p_final)

# Step 9: Clean-up Intermediate Files and Final Outputs

In [ ]:
# You'll have to delete the 'temp' directory manually.

# Remove Files
if domain == "bacteria":
    %rm $directory/output/*tree.unrooted.qza 
    %rm $directory/output/*aligned.masked.qza 
    
%rm $directory/output/*.biom 
%rm $directory/output/taxonomy.tsv
%rm $directory/output/copyrighter.tax.strings.tsv

# Separate Final Files
%mkdir $directory/final/    
%mv $directory/output/*.final.rds $directory/final/
%mv $directory/output/*.taxonomy.final.tsv $directory/final/    
%mv $directory/output/*.counts.final.tsv $directory/final/
%mv $directory/output/*.final.fasta $directory/final/
%cp $directory$metadata $directory/final/
%mv $directory/output/merged.seqID.to.rrn.final.tsv $directory/final/

# Gzip and Move Intermediate Files
!pigz -p 10 $directory/output/*.qza
!pigz -p 10 $directory/output/*.qzv
%mv $directory/output/ $directory/intermediate_files

print("Your sequences have been successfully saved to directories 'final' and 'intermediate_files'")